In [1]:
# import dependencies
import pandas as pd

File to extract individual host losses for Tableau, resolving an annoying error where I couldn't get it to plot a simple calculated field of SUM([Amount]) - SUM([Paid]).

In [2]:
# read in expenses
df = pd.read_csv('cleaned_expenses.csv')
df

,Name,Expense,Amount,Year,Category
0,Randy Detman,Key chains,150.000000,2017,Merch
1,Randy Detman,Cleaning service,60.000000,2017,Cleaning
2,Randy Detman,Carpet cleaner,50.000000,2017,Cleaning
3,Morning Glory,Brau kegs,140.000000,2017,Bar
4,Tom Brady,Plates/cups etc,50.000000,2017,Food
...,...,...,...,...,...
109,Randy Detman,Costco,178.773333,2021,Bar
110,Randy Detman,Costco,282.013333,2022,Bar
111,Randy Detman,Costco,210.306667,2023,Bar
112,Randy Detman,Costco,200.000000,2024,Bar


In [3]:
# read in reimbursements
reim_df = pd.read_csv('cleaned_reimbursements.csv')
reim_df

,Year,Name,Paid
0,2017,Randy Detman,647.00
1,2017,Watsonville,40.00
2,2017,Morning Glory,190.00
3,2017,Tom Brady,337.00
4,2018,Randy Detman,950.00
5,2018,Morning Glory,367.00
6,2018,Tom Brady,567.00
7,2019,Randy Detman,938.98
8,2019,Morning Glory,725.00
9,2019,Tom Brady,700.00


In [4]:
# group and sort sums by year
reim_sum = reim_df.groupby("Name")["Paid"].sum().reset_index()
reim_sum = reim_sum.sort_values("Paid", ascending=False)

# group and sort sums by year
df_sum = df.groupby("Name")["Amount"].sum().reset_index()
df_sum = df_sum.sort_values("Amount", ascending=False)

# merge dfs
losses = pd.merge(df_sum, reim_sum, on="Name", how="outer")
losses = losses.fillna(0)
losses["Losses"] = losses.Amount - losses.Paid
losses = losses.sort_values("Losses", ascending=False)

# Filter out people with zero losses
losses = losses[losses.Losses != 0]
losses

,Name,Amount,Paid,Losses
1,Morning Glory,4192.28,3392.50,799.78
0,Randy Detman,9421.48,8931.88,489.60
2,Tom Brady,3287.00,2854.00,433.00
9,Latvia,50.00,51.00,-1.00


In [5]:
# save to csv
losses.to_csv('tableau_losses.csv', index=False)